In [1]:
import pybullet as p
import pybullet_data
import os
import numpy as np
from matplotlib import pyplot as plt # used for saving images to file
from print_xacros import compile_to_urdf, get_symmetric_names, get_names



# urdf_names = ['nnnnnn', 'lnnnnl', 'wnnnnw', 
#               'wlnnlw', 'wwnnww', 
#              'wwllww', 'wwwwww','wwnnww']
# urdf_names = ['lnnnnl', 'wnnnnw', 'nnnnnn', 'wwnnww', 'wlnnlw', 'llnnll', 'lwnnwl']
sym_names = get_symmetric_names()
all_name_list = get_names()

# look for asymetric names not yet compiled and do those
# asym_name_list = []
# for name in all_name_list:
#     if (name not in sym_name_list):
#         asym_name_list.append(name)
# print(asym_name_list)
# random_choice = np.random.choice(len(unseen_inds), 4, replace=False)
# urdf_names = [all_name_list[unseen_inds[ind]] for ind in random_choice]
# urdf_names = asym_name_list

urdf_names = ['lwwlnl', 'wnwlnw', 'wlllll', 'lnlwnw']

robotID = None
# physicsClient = p.connect(p.GUI)
physicsClient = p.connect(p.GUI, options='--background_color_red=1.0 --background_color_green=1.0 --background_color_blue=1.0')
for urdf_name in urdf_names:
    print(urdf_name)
    p.resetSimulation() # remove all objects from the world and reset the world to initial conditions. (not needed here but kept for example)
    p.configureDebugVisualizer(p.COV_ENABLE_GUI,0,physicsClientId=physicsClient)
    p.resetDebugVisualizerCamera(0.75,180,-45,[0,0,0],physicsClientId=physicsClient) # I like this view
    p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS,0,physicsClientId=physicsClient)

    p.setGravity(0,0,-9.81)
    # planeId = p.loadURDF(os.path.join(pybullet_data.getDataPath(),
    #         "plane100.urdf"))
    startPosition=[0,0,0] 
    startOrientationRPY = [0,0,0]
    startOrientation = p.getQuaternionFromEuler(startOrientationRPY)   

    urdf_file =   'urdf/' + urdf_name  + '.urdf'
#     if not(os.path.exists(urdf_file)):
    if True:
        compile_to_urdf(urdf_name)

    robotID = p.loadURDF(
                       urdf_file, 
                       basePosition=startPosition, baseOrientation=startOrientation,
                       flags= (p.URDF_MAINTAIN_LINK_ORDER 
                        | p.URDF_USE_SELF_COLLISION
                        | p.URDF_ENABLE_CACHED_GRAPHICS_SHAPES))

    # count all joints, including fixed ones
    num_joints_total = p.getNumJoints(robotID,
                    physicsClientId=physicsClient)

    moving_joint_names = []
    moving_joint_inds = []
    moving_joint_types = []
    moving_joint_limits = []
    moving_joint_centers = []
    for j_ind in range(num_joints_total):
        j_info = p.getJointInfo(robotID, 
            j_ind)
        if j_info[2] != (p.JOINT_FIXED):
            moving_joint_inds.append(j_ind)
            moving_joint_names.append(j_info[1])
            moving_joint_types.append(j_info[2])
            j_limits = [j_info[8], j_info[9]]
            j_center = (j_info[8] + j_info[9])/2
            if j_limits[1]<=j_limits[0]:
                j_limits = [-np.inf, np.inf]
                j_center = 0
            moving_joint_limits.append(j_limits)
            moving_joint_centers.append(j_center)

    num_joints = len(moving_joint_names)
    for i in range(num_joints):
        center = moving_joint_centers[i]
        jind = moving_joint_inds[i]
        p.resetJointState( bodyUniqueId=robotID, 
            jointIndex = jind,
            targetValue= center, 
            targetVelocity = 0 )


    out = p.getCameraImage(width=640, height=480)
    rgb_array = out[2]
    image_name = urdf_name
    plt.imsave('images/' + image_name +'.png', rgb_array)



lwwlnl
compiled /home/cobracommander/modular_mbrl/urdf/lwwlnl.xacro to /home/cobracommander/modular_mbrl/urdf/lwwlnl.urdf
wnwlnw
compiled /home/cobracommander/modular_mbrl/urdf/wnwlnw.xacro to /home/cobracommander/modular_mbrl/urdf/wnwlnw.urdf
wlllll
compiled /home/cobracommander/modular_mbrl/urdf/wlllll.xacro to /home/cobracommander/modular_mbrl/urdf/wlllll.urdf
lnlwnw
compiled /home/cobracommander/modular_mbrl/urdf/lnlwnw.xacro to /home/cobracommander/modular_mbrl/urdf/lnlwnw.urdf


In [ ]:
# import pybullet as p
# import time
# import pybullet_data

# # p.connect(p.GUI)
# p.connect(p.GUI, options='--background_color_red=1.0 --background_color_green=1.0 --background_color_blue=1.0')
# p.setAdditionalSearchPath(pybullet_data.getDataPath())
# # planeId = p.loadURDF("urdf/plane_white.urdf")

# box_id = p.createCollisionShape(
#     p.GEOM_BOX,
#     halfExtents=[50, 50, 10])
# block_color = 1 
# visual_id = p.createVisualShape(
#     p.GEOM_BOX,
#     rgbaColor = [1, 1, 1, 1],
#     halfExtents=[50, 50, 10])
# planeId = p.createMultiBody(
#     baseMass=0,
#     baseCollisionShapeIndex=box_id,
#     baseVisualShapeIndex = visual_id,
#     basePosition=[0, 0, 0])

# sphereUid = p.loadURDF("sphere2.urdf", [0, 0, 2])

# redSlider = p.addUserDebugParameter("red", 0, 1, 1)
# greenSlider = p.addUserDebugParameter("green", 0, 1, 0)
# blueSlider = p.addUserDebugParameter("blue", 0, 1, 0)
# alphaSlider = p.addUserDebugParameter("alpha", 0, 1, 0.5)
# # p.changeVisualShape(planeId, -1, rgbaColor=[1, 1, 1, 1])
# while (1):
#     red = p.readUserDebugParameter(redSlider)
#     green = p.readUserDebugParameter(greenSlider)
#     blue = p.readUserDebugParameter(blueSlider)
#     alpha = p.readUserDebugParameter(alphaSlider)
#     p.changeVisualShape(sphereUid, -1, rgbaColor=[red, green, blue, alpha])
#     p.getCameraImage(320,
#                    200,
#                    flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX,
#                    renderer=p.ER_BULLET_HARDWARE_OPENGL)
#     time.sleep(0.01)